In [2]:
# 🧱 PART 1: Imports and Setup
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm
import time

# Headers to simulate browser for NSE site
headers = {
    "User-Agent": "Mozilla/5.0"
}


In [5]:
# from nsepython import nse_top_gainers

# gainers = nse_top_gainers()
# print(gainers[:5])


ImportError: cannot import name 'nse_top_gainers' from 'nsepython' (/home/user/anaconda3/envs/dev/lib/python3.11/site-packages/nsepython/__init__.py)

In [9]:
import requests
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0"
}

# NSE API URL (Live Variation)
url = "https://www.nseindia.com/api/live-analysis-variations?index=gainers"

# Create session for cookies
session = requests.Session()
session.headers.update(headers)
session.get("https://www.nseindia.com")  # initial hit for cookies

# Get JSON response
res = session.get(url)
data = res.json()

# Check available categories
available_categories = [cat[0] for cat in data['legends']]
print("Available Categories:", available_categories)

# Let's focus on NIFTY only for now
nifty_data = data.get("NIFTY", {}).get("data", [])

# Convert to DataFrame
df = pd.DataFrame(nifty_data)

# Rename and sort
df = df.rename(columns={
    "symbol": "Symbol",
    "ltp": "Last Price",
    "prev_price": "Previous Price",
    "perChange": "% Change",
    "trade_quantity": "Volume",
    "turnover": "Turnover (Cr)"
})

df = df.sort_values(by="% Change", ascending=False).reset_index(drop=True)

# Keep top 50 if available
df_top50 = df.head(50)

# Save to CSV
df_top50.to_csv("nifty_top_50_gainers.csv", index=False)

print("✅ Top 50 NIFTY gainers saved to 'nifty_top_50_gainers.csv'")


Available Categories: ['NIFTY', 'BANKNIFTY', 'NIFTYNEXT50', 'SecGtr20', 'SecLwr20', 'FOSec', 'allSec']
✅ Top 50 NIFTY gainers saved to 'nifty_top_50_gainers.csv'


In [13]:
import pandas as pd

#url = "https://www1.nseindia.com/content/indices/ind_nifty500list.csv"
url = "https://archives.nseindia.com/content/indices/ind_nifty500list.csv"


df_nifty500 = pd.read_csv(url)
df_nifty500["Symbol_NS"] = df_nifty500["Symbol"] + ".NS"

# Save for later use
df_nifty500.to_csv("nifty_500_symbols.csv", index=False)
print("✅ NIFTY 500 symbols saved to nifty_500_symbols.csv")


✅ NIFTY 500 symbols saved to nifty_500_symbols.csv


In [15]:
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm
import time

start = datetime.today() - timedelta(days=365)
end = datetime.today()

all_data = []
missed_symbols=[]
for symbol in tqdm(df_nifty500['Symbol_NS'].tolist()):
    try:
        data = yf.download(symbol, start=start, end=end, progress=False, auto_adjust=False)
        data['symbol'] = symbol.replace('.NS', '')
        data.reset_index(inplace=True)
        all_data.append(data)
        time.sleep(0.5)  # avoid throttling
    except Exception as e:
        print(f"❌ {symbol}: {e}")
        missed_symbol.append(symbol)


 11%|████▌                                     | 54/503 [01:07<07:19,  1.02it/s]
1 Failed download:
['DMART.NS']: Timeout('Failed to perform, curl: (28) Connection timed out after 10000 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
 29%|███████████▋                             | 144/503 [03:19<06:38,  1.11s/it]HTTP Error 404: 

1 Failed download:
['DUMMYRAYMN.NS']: YFTzMissingError('possibly delisted; no timezone found')
 52%|█████████████████████▏                   | 260/503 [06:28<09:33,  2.36s/it]Failed to get ticker 'JPPOWER.NS' reason: Failed to perform, curl: (28) Operation timed out after 10000 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

1 Failed download:
['JPPOWER.NS']: YFTzMissingError('possibly delisted; no timezone found')
 54%|██████████████████████                   | 271/503 [07:12<16:07,  4.17s/it]Failed to get ticker 'JYOTHYLAB.NS' reason: Failed to perform, c